In [ ]:
import os
from DailyReport import GridBuilder, DEFAULT_GRIDS_CELL_FORMAT_CONFIG, save_doc, combine_images, get_group_value, get_zone, get_report_price_info, Instrument, get_base_instrument, get_sub_instruments, combine_report, doc2pdf
from DailyReport.template import WEEKLY_TEMPLATE_PATH, DAILY_TEMPLATE_PATH
from shutil import copyfile
import math
import pandas as pd;pd.__version__

In [ ]:
root_instrument = Instrument.NZD
commentary = '''
    
'''
date = '2019-09-18'

In [ ]:
base_instrument = get_base_instrument(root_instrument)
sub_instruments = get_sub_instruments(root_instrument)
report_folder = r'report_output'
chart_folder = 'chart_output'
data_folder = 'data_output'
weekly_template_path = os.path.join(report_folder, root_instrument, 'WEEKLY.docx')
result_path = os.path.join(report_folder, root_instrument, '{0}_WEEKLY.docx'.format(root_instrument))

In [ ]:
data_path_dict = {
    'DailySinceHigh': os.path.join(data_folder, 'DailySinceHigh', root_instrument),
    'DailySinceLow': os.path.join(data_folder, 'DailySinceLow', root_instrument),
    'ER_ID2': os.path.join(data_folder, 'ER_ID2'),
    'ER_ID1': os.path.join(data_folder, 'ER_ID1'),
    'Max20': os.path.join(data_folder, 'Max20', root_instrument),
    'Min20': os.path.join(data_folder, 'Min20', root_instrument),
    'Price': os.path.join(data_folder, 'Price'),
    'RS': os.path.join(data_folder, 'RS', root_instrument),
    'RS_Change': os.path.join(data_folder, 'RS_Change', root_instrument),
    'TR_ID3': os.path.join(data_folder, 'TR_ID3'),
    'TR_ID2': os.path.join(data_folder, 'TR_ID2'),
    'TR_ID1': os.path.join(data_folder, 'TR_ID1'),
}

In [ ]:
context = {
    'instrument_group_name': root_instrument,
    'date': date,
    'commentary': commentary,
    'instrument_name': base_instrument,
}

In [ ]:
report_list = []
df = pd.concat([
        get_group_value(data_path_dict['RS'], sub_instruments, 'RS'),
        get_group_value(data_path_dict['RS_Change'], sub_instruments, 'RS_Change'),
        get_group_value(data_path_dict['Max20'], sub_instruments, 'Max20'),
        get_group_value(data_path_dict['Min20'], sub_instruments, 'Min20'),
        get_group_value(data_path_dict['DailySinceHigh'], sub_instruments, 'DailySinceHigh'),
        get_group_value(data_path_dict['DailySinceLow'], sub_instruments, 'DailySinceLow'),
    ]).T
dic = {
    'Daily': get_report_price_info(data_path_dict['Price'], base_instrument, 'Daily'),
    'Weekly': get_report_price_info(data_path_dict['Price'], base_instrument, 'Weekly'),
    'Monthly': get_report_price_info(data_path_dict['Price'], base_instrument, 'Monthly'),
}
p1_grid_context_info = GridBuilder.create_grid_context_from_df(df, DEFAULT_GRIDS_CELL_FORMAT_CONFIG, 'p1_1')
p2_grid_context_info = GridBuilder.create_grid_context_from_df(pd.DataFrame(dic), DEFAULT_GRIDS_CELL_FORMAT_CONFIG, 'p2_1')
text_context = {**p1_grid_context_info, **p2_grid_context_info, **context}
weekly_pics_list = [os.path.join(chart_folder, '{0}_{1}.png'.format(base_instrument, 'WEEKLY')), os.path.join(chart_folder, '{0}_{1}.png'.format(base_instrument, 'MONTHLY'))]
weekly_pic = os.path.join(report_folder, root_instrument, '{0}_{1}.png'.format(base_instrument, 'COMBINED'))
combine_images(weekly_pics_list, weekly_pic)
chart_context = {
    'weekly_monthly_chart': weekly_pic,
    'weekly_monthly_chart_width_percent': 1.5,
    'daily_chart': os.path.join(chart_folder, '{0}_{1}.png'.format(base_instrument, 'DAILY')),
    'daily_chart_width_percent': 1.5,
}
copyfile(WEEKLY_TEMPLATE_PATH, weekly_template_path)
save_doc({**text_context}, {**chart_context}, {}, weekly_template_path, result_path)
report_list.append(result_path)

In [ ]:
for _sub in sub_instruments:
    dic = {
        'Daily': get_report_price_info(data_path_dict['Price'], _sub, 'Daily'),
        'Weekly': get_report_price_info(data_path_dict['Price'], _sub, 'Weekly'),
        'Monthly': get_report_price_info(data_path_dict['Price'], _sub, 'Monthly'),
    };
    p1_grid_context_info = GridBuilder.create_grid_context_from_df(pd.DataFrame(dic), DEFAULT_GRIDS_CELL_FORMAT_CONFIG, 'p1_1')
    context = {
        'instrument_name': _sub,
    }
    text_context = {**p1_grid_context_info, **context}

    id1_daily_pics_list = [os.path.join(chart_folder, '{0}_{1}.png'.format(_sub, 'DAILY')), os.path.join(chart_folder, '{0}_{1}.png'.format(_sub, 'ID1'))]
    id1_daily_pic = os.path.join(report_folder, root_instrument, '{0}_{1}.png'.format(_sub, 'COMBINED'))
    combine_images(id1_daily_pics_list, id1_daily_pic)
    chart_context = {
        'id1_daily_chart': id1_daily_pic,
        'id1_daily_chart_width_percent': 1.5,
        'id2_chart': os.path.join(chart_folder, '{0}_{1}.png'.format(_sub, 'ID2')),
        'id2_chart_width_percent': 1.5,
    }

    daily_template_path = os.path.join(report_folder, root_instrument, 'DAILY.docx')
    result_path = os.path.join(report_folder, root_instrument, '{0}_DAILY.docx'.format(_sub))
    copyfile(DAILY_TEMPLATE_PATH, daily_template_path)
    save_doc({**text_context}, {**chart_context}, {}, daily_template_path, result_path)
    report_list.append(result_path)

In [ ]:
final_doc_path = daily_template_path = os.path.abspath(os.path.join(report_folder, root_instrument, '{0}.docx'.format(root_instrument)))
final_pdf_path = daily_template_path = os.path.abspath(os.path.join(report_folder, '{0}_{1}.pdf'.format(root_instrument, date)))
combine_report(report_list, final_doc_path)
doc2pdf(final_doc_path, final_pdf_path)